<a href="https://colab.research.google.com/github/LichcrazeLC/linreg/blob/main/linreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import files
uploaded = files.upload()

Saving apartmentComplexData.csv to apartmentComplexData.csv


In [ ]:
import numpy as np
import pandas as pd
import io
print(len(uploaded))
df = pd.read_csv(io.StringIO(uploaded['apartmentComplexData.csv'].decode('utf-8')))

In [ ]:
import statsmodels.api as sm
X = df[['--','---','complexAge', 'totalRooms', 'totalBedrooms', 'complexInhabitants', 'apartmentsNr','----']]
y = df['medianComplexValue']
X = sm.add_constant(X)
X.head()
est = sm.OLS(y, X).fit()
est.summary()

In [42]:
from pydantic import BaseModel

class ApartmentComplex(BaseModel):
    column0: float 
    column1: float 
    complexAge: float 
    totalRooms: float 
    totalBedrooms: float 
    complexInhabitants: float 
    apartmentsNr: float 
    column7: float
    class Config:
        schema_extra = {
            "example": {
                "column0": -122.230000, 
                "column1": 37.880000, 
                "complexAge": 41.000000,
                "totalRooms": 880.000000,
                "totalBedrooms": 129.000000,
                "complexInhabitants": 322.000000,
                "apartmentsNr": 126.000000,
                "column7": 8.325200
            }
        }


from fastapi import FastAPI
import pickle
import statsmodels.api as sm

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = est

@app.get('/')
def index():
    return {'message': 'This is the basepath of the apartment cost prediction api'}


@app.post('/predict')
def get_apartment_complex_details(data:ApartmentComplex):
    received = data.dict()
    column0 = received['column0']
    column1 = received['column1']
    complexAge = received['complexAge']
    totalRooms = received['totalRooms']
    totalBedrooms = received['totalBedrooms']
    complexInhabitants = received['complexInhabitants']
    apartmentsNr = received['apartmentsNr']
    column7 = received['column7']

    data = {'const' : 1.0,'--' : column0, '---' : column1, 'complexAge' : complexAge, 'totalRooms' : totalRooms, 'totalBedrooms' : totalBedrooms, 'complexInhabitants' : complexInhabitants, 'apartmentsNr' : apartmentsNr, '----' : column7}
    X = pd.DataFrame(data, index=[0])
    predictions = model.predict(X) 

    return {'medianComplexValue': predictions[0]}

In [43]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

server.run_app(app=app)

Public URL: NgrokTunnel: "http://0e86d9af7967.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [57]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     212.28.93.84:0 - "GET / HTTP/1.1" 200 OK
INFO:     212.28.93.84:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     212.28.93.84:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [57]
